# Max-Cut with QAOA (Qiskit)

This notebook solves a small Max-Cut instance using QAOA with Qiskit.
We also measure the **wall time** of the QAOA optimization.


In [ ]:
import time
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import StatevectorSampler


In [ ]:
# Build an Erdős–Rényi random graph
n_nodes = 6
G = nx.erdos_renyi_graph(n=n_nodes, p=0.5, seed=1)

pos = nx.spring_layout(G, seed=1)
nx.draw(G, pos, with_labels=True)
plt.title("Max-Cut instance (Qiskit)")
plt.show()


In [ ]:
def maxcut_hamiltonian_qiskit(G):
    n = len(G.nodes())
    paulis = []
    coeffs = []
    for i, j in G.edges():
        z = ["I"] * n
        z[i] = "Z"
        z[j] = "Z"
        paulis.append("".join(z))
        coeffs.append(1.0)
    return SparsePauliOp(paulis, coeffs)

H = maxcut_hamiltonian_qiskit(G)
print("Hamiltonian terms:", len(H.paulis))


In [ ]:
sampler = StatevectorSampler()
optimizer = COBYLA(maxiter=100)
qaoa = QAOA(sampler=sampler, optimizer=optimizer, reps=2)

start = time.perf_counter()
result = qaoa.compute_minimum_eigenvalue(operator=H)
end = time.perf_counter()

best_bs = result.best_measurement["bitstring"]
energy = result.optimal_value
runtime = end - start

print("Best bitstring:", best_bs)
print("Optimal energy:", energy)
print(f"Runtime: {runtime:.4f} s")


In [ ]:
def cut_value(G, bitstr):
    return sum(1 for i, j in G.edges() if bitstr[i] != bitstr[j])

cut = cut_value(G, best_bs)
print("Cut value:", cut)
